## Playground season 3, episode 07
### Start: February 14, 2023
### End: February 27, 2023

## EDA adoptation
---
Some of the EDA and convert took from: https://www.kaggle.com/code/jcaliz/ps-s03e07-a-complete-eda

In [1]:
import pandas as pd
import numpy as np
import os

In [39]:
if os.name == 'nt':
    data_path = r"D:\Coding_pratice\_Data\kaggle_playground_series\playground-series-s3e7"
else:
    data_path = '/Users/admin/_Work/Data/Practice/playground-series-s3e7'
train_csv = os.path.join(data_path, 'train.csv')
test_csv = os.path.join(data_path, 'test.csv')
hotel_csv = os.path.join(data_path, 'hotel_bookings.csv')
train_reservation_csv = os.path.join(data_path, 'Reservation Cancellation Prediction/train__dataset.csv')
test_reservation_csv = os.path.join(data_path, 'Reservation Cancellation Prediction/test___dataset.csv')
hotel_res_csv = os.path.join(data_path, 'Hotel Reservations.csv')

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
origin_train_df = pd.read_csv(train_reservation_csv)
origin_test_df = pd.read_csv(test_reservation_csv)
hotel_demand = pd.read_csv(hotel_csv)
hotel_res = pd.read_csv(hotel_res_csv)

In [42]:
print(train_df.shape)
print(test_df.shape)
print(hotel_demand.shape)
print(origin_train_df.shape)
print(hotel_res.shape)
set(origin_train_df.columns) - set(train_df.columns)
origin_train_df.head()

(42100, 19)
(28068, 18)
(119390, 32)
(18137, 18)
(36275, 19)


,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,4,0,0,0,118,2017,12,28,1,0,0,0,110.80,2,0
1,2,1,0,2,0,0,0,17,2018,4,14,1,0,0,0,145.00,0,1
2,1,0,1,5,0,0,0,349,2018,10,4,0,0,0,0,96.67,0,1
3,1,0,2,4,0,0,0,69,2018,6,12,0,0,0,0,120.00,0,1
4,2,0,0,4,1,0,0,11,2018,1,20,1,0,0,0,69.50,1,0


## Feature notes
---
* *id*: unique identifier of each booking

* *no_of_adults*: Number of adults
* *no_of_children*: Number of children
* *no_of_weekend_nights*: Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel
* *no_of_week_nights*: Number of week nights (Monday to Friday) the guest stayed or booked to stay at the hotel
* *type_of_meal_plan*: Type of meal plan booked by the customer
* *required_car_parking_space*: Does the customer require a car parking space? (0 - No, 1- Yes)
* *room_type_reserved*: Type of room reserved by the customer. The values are ciphered (encoded) by INN Hotels.
* *lead_time*: Number of days between the date of booking and the arrival date
* *arrival_year*: Year of arrival date
* *arrival_month*: Month of arrival date
* *arrival_date*: Date of the month
* *market_segment_type*: Market segment designation
* *repeated_guest*: Is the customer a repeated guest? (0 - No, 1- Yes)
* *no_of_previous_cancellations*: Number of previous bookings that were canceled by the customer prior to the current booking
* *no_of_previous_bookings_not_canceled*: Number of previous bookings not canceled by the customer prior to the current booking
* *avg_price_per_room*: Average price per day of the reservation; prices of the rooms are dynamic. (in euros)
* *no_of_special_requests*: Total number of special requests made by the customer (e.g. high floor, view from the room, etc)
* *booking_status*: Flag indicating if the booking was canceled or not.

In [5]:
train_df.room_type_reserved.value_counts()

0    29853
1     9135
3     1465
2     1131
4      320
5      190
6        6
Name: room_type_reserved, dtype: int64